In [23]:
import pandas as pd
from glob import glob

In [24]:
KWH_COST = 0.85

In [25]:
df = []
gpumetrics_paths = sorted(glob('results/*.csv'))
results_paths = sorted(glob('results/*.txt'))

for result_file, gpu_file in zip(results_paths, gpumetrics_paths):
    # Get GPU data
    gpu_data = pd.read_csv(gpu_file)
    encoder = gpu_file.split('-')[1]
    avg_power = gpu_data['Power (W)'].mean()
    time = gpu_data['Time (s)'].max()
    avg_mem = gpu_data['Memory Usage (MiB)'].mean()

    energy_kwh = avg_power * time / 1000 / 3600
    
    # Loading the results
    with open(result_file, 'r') as f:
        result = f.read()
    lines = result.split('\n')

    # Extract the relevant information
    composition = lines[1].split('-')[1]
    precision = float(lines[2].split(': ')[1])
    f1_score = float(lines[3].split(': ')[1])
    iou = float(lines[4].split(': ')[1])
    accuracy = float(lines[5].split(': ')[1])
    recall = float(lines[6].split(': ')[1])
    epoch = int(lines[7].split(': ')[1])
    time = lines[8].split(': ')[1]

    # Save results to a dict
    results = {
        'encoder': encoder,
        'composition': composition,
        'precision': precision,
        'f1_score': f1_score,
        'iou': iou,
        'accuracy': accuracy,
        'recall': recall,
        'epoch': epoch,
        'time': time,
        'avg_power': avg_power,
        'avg_mem': avg_mem,
        'energy_kwh': energy_kwh,
        'estimated cost (R$)': energy_kwh * KWH_COST * 2
    }

    df.append(results)

df = pd.DataFrame(df)
df = df.sort_values(by=['iou'], ascending=False)
df

,encoder,composition,precision,f1_score,iou,accuracy,recall,epoch,time,avg_power,avg_mem,energy_kwh,estimated cost (R$)
0,efficientnet,All+NDVI,0.8596,0.8869,0.8003,0.8948,0.9168,48,64m 35s,340.219835,6747.791604,0.366642,0.623291
2,resnet34,All+NDVI,0.8533,0.8850,0.7966,0.8896,0.9197,63,56m 38s,329.438154,3612.802068,0.311457,0.529477
1,resnet101,All+NDVI,0.8497,0.8826,0.7929,0.8872,0.9191,73,105m 51s,361.666337,8139.604115,0.638543,1.085523
